In [ ]:
# Setup
from kafka import KafkaProducer
import json
import json
import datetime 
import requests 
from bs4 import BeautifulSoup as soup
import pandas as pd
from time import sleep
from datetime import datetime
import pytz

from apscheduler.schedulers.blocking import BlockingScheduler

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

def priceTracker_AZN():
    url= 'https://it.finance.yahoo.com/quote/azn/'
    page = requests.get(url)
    html_tree = soup(page.content, 'html.parser')
    
    # Find the table 
    table_a = html_tree.find_all('div', {'class': 'Bxz(bb) D(ib) Va(t) Mih(250px)!--lgv2 W(100%) Mt(-6px) Mt(0px)--mobp Mt(0px)--mobl W(50%)!--lgv2 Mend(20px)!--lgv2 Pend(10px)!--lgv2'})[0]
    
    # Find different part of the table
    table1= table_a.find_all("span")
    table2= table_a.find_all("td")
    
    # Data
    #Open = table1[3].text
    Open = html_tree.find_all('div', {'class': 'My(6px) Pos(r) smartphone_Mt(6px)'})[0].find('span').text
    Min_max = table2[9].text
    Close = table1[1].text
    Volume = table1[11].text
    
    # Split
    Low = Min_max.split('-')[0]
    High = Min_max.split('-')[1]
    
    # Datetime
    #tz = pytz.timezone('UTC')
    tz = pytz.timezone('America/New_York')
    obj_now = datetime.now(tz)
    obj_now = obj_now.strftime("%Y/%m/%d %H:%M:%S")
    
    # Replace
    obj_now = pd.to_datetime(obj_now)
    Open = Open.replace(',', '.')
    High = High.replace(',', '.')
    Low = Low.replace(',', '.')
    Close = Close.replace(',', '.')
    Volume = Volume.replace('.', '')
    
    # dict
    price = {'Datetime': obj_now,
        'Open': Open,
        'High': High,
        'Low': Low,
        'Close': Close,
        'Volume': Volume
       }
    
    return price
        
producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
  value_serializer=lambda v: json.dumps(v, indent=4, sort_keys=True, default=str).encode("utf-8")) 

# Scheduler
sched = BlockingScheduler()


def job_function():
    data = priceTracker_AZN()
    producer.send(topic='azn_scrape', value=data)
    
try:
    sched.add_job(job_function, 'cron', day_of_week='mon-fri', hour='13-20', minute=30)
    sched.start()

except:
    print("An exception occurred")
    #pass